In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
freqData=pd.read_csv("C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/Data folder/Frequency_Envelop.csv")
freqData

In [ ]:
correlation_matrix = freqData.corr()
correlation_matrix



In [ ]:
col = []
for magnitude in freqData["Mag"]:
    if 6 <= magnitude <= 7:
        col.append('g')
    elif 7 < magnitude <= 8:
        col.append('r')
    elif 5 <= magnitude < 6:
        col.append('b')
    elif 4 <= magnitude < 5:
        col.append('y')
    else:
        col.append('m')




In [ ]:


# import matplotlib.pyplot as plt
# import numpy as np

# # Sample data
# x = np.random.rand(100)
# y = np.random.rand(100)
# colors = np.random.rand(100)  # Random colors for each point
# sizes = 1000 * np.random.rand(100)  # Random sizes for each point

# # Single color
# # plt.scatter(x, y, color='blue')

# # Sequence of colors
# # plt.scatter(x, y, c=colors)  # c argument specifies the sequence of colors

# # Colormap
# plt.scatter(x, y, c=colors, cmap='viridis')  # cmap argument specifies the colormap
# print(colors)

# plt.colorbar()  # Add colorbar for the last plot
# plt.show()

In [ ]:
freqData

In [ ]:
import math as mt

Fseries = Y_val[0:6]
fig, ax = plt.subplots(ncols=2, nrows=mt.ceil(
    len(Fseries)/2), figsize=(25,7.5*(len(Fseries)/2)))
ax = ax.flatten()

for i in range(len(Fseries)):
    ax[i].scatter(freqData['Rjb'],
                  freqData[Fseries[i]], c=col, alpha=0.5,marker="o", s=4 )
    ax[i].set_xlabel('Rjb')
    ax[i].set_ylabel(Fseries[i])
    ax[i].set_xscale("log")
    # ax[i].set_yscale("log")
    ax[i].set_title(f'Rjb vs {Fseries[i]}')

plt.show()

In [ ]:
import math as mt
# Plot the scatter plot with colors

fig, ax = plt.subplots(ncols=2, nrows=mt.ceil(
    len(X_val)/2), figsize=(25, 7.5*(len(X_val)/2)))
ax = ax.flatten()
# ws="TimeI_  7"
ws="FreI_  8"
for i in range(len(X_val)):
    ax[i].scatter(freqData[X_val[i]], freqData[ws], c="b",marker="o", alpha=0.5, s=4)
    # ax[i].set_xscale("log")
    # ax[i].set_yscale("log")
    ax[i].set_ylabel(ws+" S")
    ax[i].set_xlabel(X_val[i])
    ax[i].set_title(f'{X_val[i]} vs {ws} s')

plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, models,optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np



#Normalizing the data
scalerX = MinMaxScaler()
scalerY = MinMaxScaler()
X_normalized = scalerX.fit_transform(Xreq)
Y_normalized = scalerY.fit_transform(Yreq)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_normalized, Y_normalized, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Define the neural network architecture
model = models.Sequential()

# Input layer with 4 neurons
model.add(layers.InputLayer(input_shape=(X_train.shape[1],),name="inputLayer"))

# Hidden layers with 6 neurons each (2 hidden layers)
# for i in range(2):
#     model.add(layers.Dense(6, activation='relu', name=f"{i}_layer"))
model.add(layers.Dense(25, activation='relu', name="hidden_layer_1"))
model.add(layers.Dense(25, activation='relu', name="hidden_layer_2"))
# model.add(layers.Dense(25, activation='relu', name="hidden_layer_2"))

# Output layer with 51 neurons for multi-class regression
model.add(layers.Dense(y_train.shape[1], activation='linear',name="Output"))

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),  # You can choose other optimizers as well
              loss='mean_squared_error',  # Use appropriate loss function for regression
              metrics=['mae'])  # Use appropriate metrics for regression

# Display the model summary
model.summary()
# early_stopping = EarlyStopping(
#     monitor='val_loss', patience=10, restore_best_weights=True)


# Train the model using the training set and validate on the validation set
# history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_val, y_val),callbacks=[early_stopping])
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


In [ ]:
X_train.shape
len(y_train)

In [ ]:
plt.plot(history.history['loss'])
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.show()

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation MAE
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
#saving the model 
model.save("C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/Data folder/Model.keras")


In [ ]:
# Load the model
model = models.load_model("C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/Data folder/Model.keras")

# Make predictions on new data
predictions = model.predict(X_test)


# Calculate evaluation metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
rsquared = r2_score(y_test, predictions)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", rsquared)

X_test=scalerX.inverse_transform(X_test)
predictions=scalerY.inverse_transform(predictions)
y_test=scalerY.inverse_transform(y_test)
predictions_df= pd.DataFrame(np.array(predictions),columns=Yreq.columns)
predictions_df

In [ ]:
Fseries = list(predictions_df.columns)
for i in range(predictions.shape[1]):
    plt.scatter(y_test[:, i], predictions[:, i], label=Fseries[i])
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Scatter plot for {Fseries[i]}')
    plt.legend()
    plt.grid(True)
    plt.show()
    # plt.savefig(f"C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/PredVsActualPlot/Scatter plot for {Fseries[i]}.png")
# print(predictions_df.shape)
# print(y_test.shape)

In [ ]:
#calculating the residual plot for the model
residuals = y_test - predictions
residuals_df = pd.DataFrame(residuals, columns=Yreq.columns)

for i in range(len(Fseries)):
    plt.scatter(y_test[:, i], residuals[:, i],c='b', alpha=0.5,marker="o", s=4 )
    plt.xlabel('Actual Values')
    plt.ylabel('Residuals')
    plt.title(f'Residual plot for {Fseries[i]}')
    plt.grid(True)
    plt.show()
    # plt.savefig(f"C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/ResidualPlot/Residual plot for {Fseries[i]}.png")



In [ ]:
#plotting the residual distribution with respect to prdicted values
for i in range(len(Fseries)):
    plt.scatter(predictions[:, i], residuals[:, i],c='b', alpha=0.5,marker="o", s=4 )
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title(f'Residual plot for {Fseries[i]}')
    plt.grid(True)
    plt.show()
    # plt.savefig(f"C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/ResidualPlot/Residual plot for {Fseries[i]}.png")

In [ ]:
#Plotting the residual distribution with respect to Rjb
for i in range(len(Fseries)):
    plt.scatter(X_test[:, 1], residuals[:, i],c='b', alpha=0.5,marker="o", s=4 )
    plt.xlabel('Rjb')
    plt.ylabel('Residuals')
    plt.title(f'Residual plot for {Fseries[i]}')
    plt.grid(True)
    plt.show()
    # plt.savefig(f"C:/Users/adity/OneDrive/Desktop/Sixth Semester/CE6018 Seismic Data Analytics/Program/Week8/ResidualPlot/Residual plot for {Fseries[i]}.png")